In [3]:
import sys
sys.path.insert(1, '/home/jupyter/ospc/src')
from churn_utils import *
from utils import *

## Write query

In [17]:
query = """WITH q1 AS
           (select usage_date,
                  account_id,
                  SUM(acct_ttl_unit_hours) as acct_ttl_unit_hours
           from cloud_usage.rpt_tbl_server_uptime_acctdaily
           where account_id = 615281
           GROUP BY usage_date,
                    account_id
           ORDER BY usage_date ASC),
           
      q2 AS (
      SELECT 
      q1.*,
       AVG(acct_ttl_unit_hours) OVER (ORDER BY usage_date ROWS BETWEEN 5 PRECEDING AND CURRENT ROW) as a5,
       AVG(acct_ttl_unit_hours) OVER (ORDER BY usage_date ROWS BETWEEN 10 PRECEDING AND CURRENT ROW) as a10,
       AVG(acct_ttl_unit_hours) OVER (ORDER BY usage_date ROWS BETWEEN 15 PRECEDING AND CURRENT ROW) as a15,
       STDDEV(acct_ttl_unit_hours) OVER (ORDER BY usage_date ROWS BETWEEN 5 PRECEDING AND CURRENT ROW) as s5,
       STDDEV(acct_ttl_unit_hours) OVER (ORDER BY usage_date ROWS BETWEEN 10 PRECEDING AND CURRENT ROW) as s10,
       STDDEV(acct_ttl_unit_hours) OVER (ORDER BY usage_date ROWS BETWEEN 15 PRECEDING AND CURRENT ROW) as s15,
       FROM q1
      )
           
SELECT usage_date,
       acct_ttl_unit_hours,
       CASE WHEN s5 = 0 THEN 0 ELSE SAFE_DIVIDE(acct_ttl_unit_hours-a5,s5) END as z5,
       CASE WHEN s10 = 0 THEN 0 ELSE SAFE_DIVIDE(acct_ttl_unit_hours-a10,s10) END as z10,
       CASE WHEN s15 = 0 THEN 0 ELSE SAFE_DIVIDE(acct_ttl_unit_hours-a15,s15) END as z15
FROM q2
      WHERE s5 IS NOT NULL
      """

In [18]:
dataset = fetch_data(source_type='query',source_name=query)

In [19]:
dataset.to_csv('timeseries_data.csv')